In [364]:
# super graph do narumi e da mel
import networkx as nx
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import matplotlib as mpl
import matplotlib.cm as cm

G = nx.Graph()
G.add_edge('Next', 'BB')
G.add_edge('Next', 'Nubank')
G.add_edge('Next', 'Itau')
G.add_edge('Next', 'XP Investimentos')
G.node['Next']['valor'] = 0
G.node['BB']['valor']= 450000
G.node['Nubank']['valor'] = 10000
G.node['Itau']['valor'] = 100
G.node['XP Investimentos']['valor'] = 89000

vmax = 0
for g in G.nodes:
    if G.nodes[g]['valor'] > vmax:
        vmax = G.nodes[g]['valor']
norm = mpl.colors.Normalize(vmin=0, vmax=vmax)
cmap = cm.hot
m = cm.ScalarMappable(norm=norm, cmap=cmap)


#nx.draw(G, with_labels=True)  
#plt.draw()  

pos = nx.circular_layout(G)
position = pos
G.add_nodes_from([(k[0], {'pos':[k[1][0],k[1][1]]}) for k in pos.items()])

In [371]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Montante transferido (R$)',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

annotations = []
for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    annotations.append(
        dict(
            x=x,
            y=y+0.10,
            xref='x',
            yref='y',
            text=node,
            showarrow=False,
            ax=0,
            ay=20
        )        
    )
    
layout = go.Layout(
    showlegend=False,
    annotations=annotations,
    title='Super Graph',
    titlefont=dict(size=16),
    hovermode='closest',
    margin=dict(b=20,l=5,r=5,t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

In [372]:
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color'] =  m.to_rgba(G.nodes[list(G.nodes.keys())[node]]['valor'])
    node_info = 'R$ transferidos: '+str(G.nodes[list(G.nodes.keys())[node]]['valor'])
    node_trace['text']+=tuple([node_info])

In [373]:
fig = go.Figure(data=[edge_trace, node_trace],layout=layout)
iplot(fig, filename='networkx')